### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
learning_rate = 0.0001
batch_size = 1
N = 256
max_pool = 256
regularization = 0.0001

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "DPCL_train_front"
config_model["smooth_size"] = 10
idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_fine_tuning"

In [5]:
from models.adapt import Adapt

full_id = 'shiny-frog-0617'+idd

folder='DPCL_fine_tuning'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)

from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.sepNet.output = model.sepNet.prediction
    model.create_saver()
    model.restore_model('DPCL_train_front', full_id)
    model.sepNet.output = model.sepNet.separate
    model.separator
    model.back
    model.cost
    model.optimize
    model.tensorboard_init()



ID : AdaptiveNet-white-wood-3867
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/DPCL_train_front/AdaptiveNet-shiny-frog-0617-N=256--alpha=0.0001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.0001--smooth_size=10--type=DPCL_train_front-/model.ckpt
Tensor("separate/Reshape_3:0", shape=(?, ?, 256, 1), dtype=float32)
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 10, 1, 1) dtype=float32_ref>, <tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_l

## Connect DPCL model to the front end and back end

In [6]:
with model.graph.as_default():
    uninit =[v for v in tf.global_variables() if v.name.split(':')[0] in set(model.sess.run(tf.report_uninitialized_variables()))]
    print uninit
    init = tf.variables_initializer(uninit)

[<tf.Variable 'optimize/beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'optimize/beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'optimize/conv/W/Adam:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'optimize/conv/W/Adam_1:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'optimize/smooth/smoothing_filter/Adam:0' shape=(1, 10, 1, 1) dtype=float32_ref>, <tf.Variable 'optimize/smooth/smoothing_filter/Adam_1:0' shape=(1, 10, 1, 1) dtype=float32_ref>]


# Model creation

# Pretraining the model 

In [7]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 1.05886
DAS model saved at iteration number  0  with cost =  1.05886
Step # 1  loss= 1.14568
Step # 2  loss= 1.23254
Step # 3  loss= 1.26422
Step # 4  loss= 0.921672
Step # 5  loss= 1.44156
Step # 6  loss= 1.86411
Step # 7  loss= 1.26247
Step # 8  loss= 1.11814
Step # 9  loss= 1.01655
Step # 10  loss= 1.39125
Step # 11  loss= 1.36994
Step # 12  loss= 1.31438
Step # 13  loss= 1.5192
Step # 14  loss= 1.60661
Step # 15  loss= 1.28589
Step # 16  loss= 0.871381
Step # 17  loss= 1.09316
Step # 18  loss= 1.47585
Step # 19  loss= 1.60712
Step # 20  loss= 1.0035
DAS model saved at iteration number  20  with cost =  1.0035
Step # 21  loss= 1.41269
Step # 22  loss= 1.01978
Step # 23  loss= 1.1543
Step # 24  loss= 1.1553
Step # 25  loss= 0.91606
Step # 26  loss= 1.01329
Step # 27  loss= 1.15096
Step # 28  loss= 0.727809
Step # 29  loss= 1.45577
Step # 30  loss= 0.926761
Step # 31  loss= 1.06742
Step # 32  loss= 1.12482
Step # 33  loss= 1.66813
Step # 34  loss= 1.1244
Step # 35  los

KeyboardInterrupt: 